In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import pathlib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchinfo import summary
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

from dataset import CriteoAdDataset
from model import DeepFM

## Dataset

In [ ]:
data_dir = pathlib.Path("data/criteo-ad-data")
train_dataset = CriteoAdDataset(data_dir, type="train", nums=10000)
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_dataset = CriteoAdDataset(data_dir, type="val", nums=10000)
val_dataloader = DataLoader(val_dataset, batch_size=10, shuffle=False)


In [ ]:
label, count_features, category_features = next(iter(train_dataloader))
label.shape, count_features.shape, category_features.shape


## Model

In [ ]:
embedding_dims = 20
category_feature_names = train_dataset.category_feature_columns
category_cardinalities = train_dataset.category_cardinalities

dense_embedding_in_features = len(train_dataset.count_feature_columns)
dense_embedding_hidden_features = 30
deep_layer_out_features = 10

model = DeepFM(
    embedding_dims=embedding_dims,
    category_cardinalities=category_cardinalities,
    dense_embedding_in_features=dense_embedding_in_features,
    dense_embedding_hidden_features=dense_embedding_hidden_features,
    deep_layer_out_features=deep_layer_out_features,
)


In [ ]:
summary(
    model,
    count_features=count_features,
    category_features=category_features,
    category_feature_names=category_feature_names,
)


## Train

In [ ]:
def train(
    train_dataloader: DataLoader,
    model: nn.Module,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    log_interval: int = 10000,
):
    size = len(train_dataloader.dataset)
    sum_loss = 0
    for batch, (labels, count_features, category_features) in tqdm(
        enumerate(train_dataloader, 1), total=len(train_dataloader), unit="iter"
    ):
        model.train()
        logits = model(
            count_features=count_features,
            category_features=category_features,
            category_feature_names=category_feature_names,
        )
        loss = criterion(logits.squeeze(), labels.float())
        optimizer.zero_grad()
        loss.backward()
        # to prevent gradient explosion
        nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()

        sum_loss += loss.item()

        if batch % log_interval == 0:
            mean_loss = sum_loss / log_interval
            tqdm.write(f"loss: {mean_loss:>7f} [{batch * len(labels):>5d}/{size:>5d}]")
            sum_loss = 0


def eval(dataloader: DataLoader, model: nn.Module, criterion: nn.Module):
    model.eval()
    sum_loss = 0
    num_batches = len(dataloader)
    labels_list = []
    preds_list = []
    with torch.no_grad():
        for labels, count_features, category_features in tqdm(
            dataloader, total=len(dataloader), unit="iter"
        ):
            logits = model(
                count_features=count_features,
                category_features=category_features,
                category_feature_names=category_feature_names,
            )
            preds = F.sigmoid(logits.squeeze())
            sum_loss += criterion(preds, labels.float()).item()

            labels_list.extend(labels.tolist())
            preds_list.extend(preds.tolist())

    mean_loss = sum_loss / num_batches
    accuracy = accuracy_score(labels_list, np.round(np.asarray(preds_list)))
    roc_auc = roc_auc_score(labels_list, preds_list)
    pr_auc = average_precision_score(labels_list, preds_list)

    return mean_loss, accuracy, roc_auc, pr_auc


In [ ]:
epochs = 3
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm(range(epochs), unit="epoch"):
    train(train_dataloader, model, criterion, optimizer, log_interval=2000)
    mean_loss, accuracy, roc_auc, pr_auc = eval(train_dataloader, model, criterion)
    tqdm.write(f"Train | avg loss: {mean_loss:.4f}, accuracy: {accuracy:.4f}, roc_auc: {roc_auc:.4f}, pr_auc: {pr_auc:.4f}")
    mean_loss, accuracy, roc_auc, pr_auc = eval(val_dataloader, model, criterion)
    tqdm.write(f"Val | avg loss: {mean_loss:.4f}, accuracy: {accuracy:.4f}, roc_auc: {roc_auc:.4f}, pr_auc: {pr_auc:.4f}")
